In [20]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
%matplotlib inline

In [2]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target

In [3]:
URM_all = load_urm()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.85)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 4306 (10.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 3214 ( 7.7%) Users that have less than 1 test interactions


In [4]:
URM_train

<41629x24507 sparse matrix of type '<class 'numpy.float64'>'
	with 759946 stored elements in Compressed Sparse Row format>

In [5]:
n_users, n_items = URM_train.shape

In [8]:
item_item_S = np.zeros((n_items, n_items), dtype = float)

In [9]:
URM_train_coo = URM_train.tocoo()
sample_index = np.random.randint(URM_train_coo.nnz)

In [10]:
user_id = URM_train_coo.row[sample_index]
item_id = URM_train_coo.col[sample_index]
true_rating = URM_train_coo.data[sample_index]

In [11]:
predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]

In [13]:
prediction_error = true_rating - predicted_rating

In [15]:
items_in_user_profile = URM_train[user_id].indices
ratings_in_user_profile = URM_train[user_id].data

item_item_S[items_in_user_profile,item_id]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
URM_train_coo = URM_train.tocoo()
item_item_S = np.zeros((n_items, n_items), dtype = np.float)

learning_rate = 1e-6

loss = 0.0
start_time = time.time()
for sample_num in range(100000):

    # Randomly pick sample
    sample_index = np.random.randint(URM_train_coo.nnz)

    user_id = URM_train_coo.row[sample_index]
    item_id = URM_train_coo.col[sample_index]
    true_rating = URM_train_coo.data[sample_index]

    # Compute prediction
    predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]

    # Compute prediction error, or gradient
    prediction_error = true_rating - predicted_rating
    loss += prediction_error**2

    # Update model, in this case the similarity
    items_in_user_profile = URM_train[user_id].indices
    ratings_in_user_profile = URM_train[user_id].data
    item_item_S[items_in_user_profile,item_id] += learning_rate * prediction_error * ratings_in_user_profile

    # Print some stats
    if (sample_num +1)% 5000 == 0:
        elapsed_time = time.time() - start_time
        samples_per_second = (sample_num +1)/elapsed_time
        print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/(sample_num +1), samples_per_second))

C:\Users\Luca\AppData\Local\Temp\ipykernel_5856\1578646889.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  item_item_S = np.zeros((n_items, n_items), dtype = np.float)


Iteration 5000 in 9.92 seconds, loss is 1.00. Samples per second 504.23
Iteration 10000 in 18.85 seconds, loss is 1.00. Samples per second 530.64
Iteration 15000 in 25.05 seconds, loss is 1.00. Samples per second 598.71
Iteration 20000 in 31.42 seconds, loss is 1.00. Samples per second 636.63
Iteration 25000 in 38.18 seconds, loss is 1.00. Samples per second 654.85
Iteration 30000 in 44.68 seconds, loss is 1.00. Samples per second 671.47
Iteration 35000 in 50.90 seconds, loss is 1.00. Samples per second 687.63
Iteration 40000 in 57.81 seconds, loss is 1.00. Samples per second 691.98
Iteration 45000 in 65.05 seconds, loss is 1.00. Samples per second 691.75
Iteration 50000 in 71.81 seconds, loss is 1.00. Samples per second 696.24
Iteration 55000 in 78.43 seconds, loss is 1.00. Samples per second 701.27
Iteration 60000 in 83.70 seconds, loss is 1.00. Samples per second 716.86
Iteration 65000 in 90.18 seconds, loss is 1.00. Samples per second 720.80
Iteration 70000 in 96.13 seconds, loss i

In [22]:
def train_one_epoch(URM_train, item_item_S, learning_rate):

    URM_train_coo = URM_train.tocoo()

    loss = 0.0
    start_time = time.time()
    for sample_num in range(URM_train.nnz):

        # Randomly pick sample
        sample_index = np.random.randint(URM_train_coo.nnz)

        user_id = URM_train_coo.row[sample_index]
        item_id = URM_train_coo.col[sample_index]
        true_rating = URM_train_coo.data[sample_index]

        # Compute prediction
        predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]

        # Compute prediction error, or gradient
        prediction_error = true_rating - predicted_rating
        loss += prediction_error**2

        # Update model, in this case the similarity
        items_in_user_profile = URM_train[user_id].indices
        ratings_in_user_profile = URM_train[user_id].data
        item_item_S[items_in_user_profile,item_id] += learning_rate * prediction_error * ratings_in_user_profile

        # Print some stats
        if (sample_num +1)% 5000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = (sample_num +1)/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/(sample_num +1), samples_per_second))

            # Stop training because this implementation is too slow
            print("\tStopping the epoch early because this implementation is too slow")
            return item_item_S

    return item_item_S

In [23]:
n_items = URM_train.shape[1]
learning_rate = 1e-6

item_item_S = np.zeros((n_items, n_items), dtype = np.float)

for n_epoch in range(5):
    item_item_S = train_one_epoch(URM_train, item_item_S, learning_rate)

C:\Users\Luca\AppData\Local\Temp\ipykernel_5856\2493220389.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  item_item_S = np.zeros((n_items, n_items), dtype = np.float)


Iteration 5000 in 8.92 seconds, loss is 1.00. Samples per second 560.84
	Stopping the epoch early because this implementation is too slow
Iteration 5000 in 5.47 seconds, loss is 1.00. Samples per second 914.71
	Stopping the epoch early because this implementation is too slow
Iteration 5000 in 7.17 seconds, loss is 1.00. Samples per second 697.01
	Stopping the epoch early because this implementation is too slow
Iteration 5000 in 5.67 seconds, loss is 1.00. Samples per second 881.60
	Stopping the epoch early because this implementation is too slow
Iteration 5000 in 6.44 seconds, loss is 1.00. Samples per second 776.03
	Stopping the epoch early because this implementation is too slow


In [24]:
item_item_S

array([[1.e-06, 0.e+00, 0.e+00, ..., 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 1.e-06, 0.e+00, ..., 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, ..., 0.e+00, 0.e+00, 0.e+00],
       ...,
       [0.e+00, 0.e+00, 0.e+00, ..., 1.e-06, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, ..., 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, ..., 0.e+00, 0.e+00, 0.e+00]])

In [28]:


recommender = ItemKNNCFRecommender(URM_train)
recommender.fit(shrink=50.0, topK=100)

Similarity column 24507 (100.0%), 456.47 column/sec. Elapsed time 53.69 sec


## Submissions

In [31]:
test_users = pd.read_csv('..\Dataset\data_target_users_test.csv')
test_users

FileNotFoundError: [Errno 2] No such file or directory: 'Dataset\\data_target_users_test.csv'

In [ ]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user, at=10))

In [ ]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('Submission_10933934.csv', index=False)